In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path

    local_train_angry_dir: Path
    local_train_disgust_dir: Path
    local_train_fear_dir: Path
    local_train_happy_dir: Path
    local_train_sad_dir: Path
    local_train_surprise_dir: Path
    local_train_nuetral_dir: Path

    local_val_angry_dir: Path
    local_val_disgust_dir: Path
    local_val_fear_dir: Path
    local_val_happy_dir: Path
    local_val_sad_dir: Path
    local_val_surprise_dir: Path
    local_val_nuetral_dir: Path

    local_test_angry_dir: Path
    local_test_disgust_dir: Path
    local_test_fear_dir: Path
    local_test_happy_dir: Path
    local_test_sad_dir: Path
    local_test_surprise_dir: Path
    local_test_nuetral_dir: Path

    local_train_file: Path
    local_val_file: Path
    local_test_file: Path


In [4]:
import os
import sys
import pandas as pd
from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging

from cnnClassifier.constants import *
from cnnClassifier.utils import pixel_to_matrix, save_object, create_directories, read_yaml

2023-08-03 16:13:56.081058: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 16:13:56.134267: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 16:13:56.135586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 16:13:57.327382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation             

        create_directories([config.root_dir,
                            config.local_train_angry_dir,
                            config.local_train_disgust_dir,
                            config.local_train_fear_dir,
                            config.local_train_happy_dir,
                            config.local_train_sad_dir,
                            config.local_train_surprise_dir,
                            config.local_train_nuetral_dir,
                            config.local_val_angry_dir,
                            config.local_val_disgust_dir,
                            config.local_val_fear_dir,
                            config.local_val_happy_dir,
                            config.local_val_sad_dir,
                            config.local_val_surprise_dir,
                            config.local_val_nuetral_dir,
                            config.local_test_angry_dir,
                            config.local_test_disgust_dir,
                            config.local_test_fear_dir,
                            config.local_test_happy_dir,
                            config.local_test_sad_dir,
                            config.local_test_surprise_dir,
                            config.local_test_nuetral_dir]
                            )

        data_trnsformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            local_train_angry_dir= config.local_train_angry_dir,
            local_train_disgust_dir= config.local_train_disgust_dir,
            local_train_fear_dir= config.local_train_fear_dir,
            local_train_happy_dir= config.local_train_happy_dir,
            local_train_sad_dir= config.local_train_sad_dir,
            local_train_surprise_dir= config.local_train_surprise_dir,
            local_train_nuetral_dir= config.local_train_nuetral_dir,

            local_val_angry_dir = config.local_val_angry_dir,
            local_val_disgust_dir= config.local_val_disgust_dir,
            local_val_fear_dir= config.local_val_fear_dir,
            local_val_happy_dir= config.local_val_happy_dir,
            local_val_sad_dir= config.local_val_sad_dir,
            local_val_surprise_dir= config.local_val_surprise_dir,
            local_val_nuetral_dir= config.local_val_nuetral_dir,

            local_test_angry_dir= config.local_test_angry_dir,
            local_test_disgust_dir= config.local_test_disgust_dir,
            local_test_fear_dir= config.local_test_fear_dir,
            local_test_happy_dir= config.local_test_happy_dir,
            local_test_sad_dir= config.local_test_sad_dir,
            local_test_surprise_dir= config.local_test_surprise_dir,
            local_test_nuetral_dir= config.local_test_nuetral_dir,            
            
            local_train_file = self.config.data_ingestion.local_train_file,
            local_val_file = self.config.data_ingestion.local_val_file,
            local_test_file = self.config.data_ingestion.local_test_file,           

        )

        return data_trnsformation_config

In [6]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import Pipeline

In [7]:
from keras.preprocessing.image import save_img

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_transformer_object(self):
        try:            
            face_recognition_pipeline = Pipeline(
                steps=[
                    ('pixel_to_image', 
                    FunctionTransformer(pixel_to_matrix, kw_args={'img_height':48, 'img_width':48}))
                ]
            )
            return face_recognition_pipeline            
        
        except Exception as e:
            raise CustomException(e, sys)
    
    
    def initiate_data_transformation(self):
        if not os.path.exists(self.config.local_train_file):
            logging.info(f"WARNING: {self.config.local_train_file} does not exist!")
        
        elif not os.path.exists(self.config.local_val_file):
            logging.info(f"WARNING: {self.config.local_val_file} does not exist!")   

        elif not os.path.exists(self.config.local_test_file):
            logging.info(f"WARNING: {self.config.local_test_file} does not exist!")              
        
        else:  
            
            train_df = pd.read_csv(self.config.local_train_file)
            val_df = pd.read_csv(self.config.local_val_file)
            test_df = pd.read_csv(self.config.local_test_file)
            logging.info('Read train, validation and test data completed')

            logging.info("Obtaining preprocessing object")
            preprocessing_obj=self.get_data_transformer_object()    

            emotion_path = {0: self.config.local_train_angry_dir,
                            1: self.config.local_train_disgust_dir,
                            2: self.config.local_train_fear_dir,
                            3: self.config.local_train_happy_dir,
                            4: self.config.local_train_sad_dir,
                            5: self.config.local_train_surprise_dir,
                            6: self.config.local_train_nuetral_dir}                 

     
            logging.info(f"Applying preprocessing object on train, validation and test dataframes")
            emotions, images = preprocessing_obj.fit_transform(train_df)
            i=1
            for emotion, image in zip(emotions, images):
                path_config = emotion_path[emotion]
                path_file = os.path.join(path_config, str(i)+'.png')
                save_img(path_file, image)
                i+=1
            logging.info(f"Train set is saved as .png")  
            
            for emotion, path in emotion_path.items():
                path = path.replace('train', 'validation')
                emotion_path[emotion] = path       
            
            emotions, images = preprocessing_obj.fit_transform(val_df)  
            i=1
            for emotion, image in zip(emotions, images):
                path_config = emotion_path[emotion]
                path_file = os.path.join(path_config, str(i)+'.png')
                save_img(path_file, image)
                i+=1        
            logging.info(f"Validation set is saved as .png")
            
            for emotion, path in emotion_path.items():
                path = path.replace('validation', 'test')
                emotion_path[emotion] = path
            emotions, images = preprocessing_obj.transform(test_df) 
            i=1
            for emotion, image in zip(emotions, images):
                path_config = emotion_path[emotion]
                path_file = os.path.join(path_config, str(i)+'.png')
                save_img(path_file, image)
                i+=1  
            logging.info(f"Test set is saved as .png")   

In [8]:
try:
    config = configurationManeger()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.initiate_data_transformation()

except Exception as e:
    raise CustomException(e, sys)